In [3]:
import pandas as pd
import os
import pickle

from model import Transformer # this is the transformer.py file
import torch
from torch import nn
import numpy as np

In [9]:
import netCDF4 as nc

# Open the .nc file
ds = nc.Dataset('data/precipitation_data.nc')

ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: GPCP 1x1 Precipitation Monitoring
    Conventions: CF-1.0
    history: CAS Jul 2021
    platform: Observation
    source: PSL
    References: https://www.psl.noaa.gov/data/gridded/data.gpcc.html
    Original_Source: http://www.dwd.de/en/FundE/Klima/KLIS/int/GPCC/GPCC.htm
 is the webpage and the data is at ftp://ftp.dwd.de/pub/data/gpcc/download.html
    Reference: Users of the data sets are kindly requested 
to give feed back and to refer to following GPCC publications:
Rudolf, B., H. Hauschild, W. Rueth and U. Schneider (1994)
: Terrestrial Precipitation Analysis: Operational Method and Required
 Density of Point Measurements. In: Global Precipitations and
 Climate Change (Ed. M. Desbois, F. Desalmond), NATO ASI Series I
, Vol. 26, Springer-Verlag, p. 173-186.
Rudolf, B., T. Fuchs, U. Schneider and A. Meyer-Christoffer (2003):
, Introduction of the Global Precipitation Climatolo